In [0]:
import cv2
im = cv2.imread("01_manual1.gif", mode = 'RGB')
print((im))

In [0]:
from matplotlib.image import imread
img = imread('01_manual1.gif')
print((img))

In [0]:
import numpy as np
from skimage.util.shape import view_as_windows
A = img
patch_shape = (8, 8)
B = view_as_windows(A, patch_shape)
C = B.reshape(-1, patch_shape[0] * patch_shape[1])[::8]
C.shape

In [0]:
from scipy.cluster.vq import kmeans2
import numpy as np
from scipy.cluster.vq import kmeans2
from scipy import ndimage as ndi
import matplotlib.pyplot as plt

from skimage import data
from skimage import color
from skimage.util.shape import view_as_windows
from skimage.util import montage
n_filters = 49

fb1, _ = kmeans2(C, n_filters, minit='points')
fb1 = fb1.reshape((-1,) + patch_shape)
fb1_montage = montage(fb1, rescale_intensity=True)

In [0]:
import skimage
skimage.io.imshow(C)

In [0]:
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image
# Use the array data from the first image in this dataset:
from matplotlib.image import imread

one_image = imread('01_manual1.gif')
# one_image = load_sample_image('01_manual1.gif')
print('Image shape: {}'.format(one_image.shape))

patches = image.extract_patches_2d(one_image, (300, 300))
print('Patches shape: {}'.format(patches.shape))

# Here are just two of these patches:
print(patches[1])
print(patches[800])
import skimage
skimage.io.imshow_collection(patches)


In [0]:
from skimage.util import view_as_windows
import numpy as np

from matplotlib.image import imread
import matplotlib.pyplot as plt
one_image = plt.imread('34_training.tif')
# image_data = ndimage.imread(image_file).astype(float)

In [0]:
print(one_image)

<PIL.TiffImagePlugin.TiffImageFile image mode=RGB size=565x584 at 0x1565C059108>


In [0]:
import scipy

In [0]:
from PIL import Image
import numpy as np
import cv2
gif = cv2.VideoCapture('01_manual1.gif')
ret,frame = gif.read()
image_data = Image.fromarray(frame)
image_data = np.array(image_data)


In [0]:
sizex, sizey, *t = image_data.shape

In [0]:
sizex

584

In [0]:
WindowSize = 23 #patch size
ws2 = WindowSize // 2
dimensions = image_data.shape
SizeX = dimensions[0]
SizeY = dimensions[1]


In [0]:
image_data[1,1,1]

0

In [0]:
for y in range(ws2,SizeY-ws2):
    for x in range(ws2,SizeX-ws2):
        imtest[x-ws2,0] = image_data[y-ws2:y+ws2+1, x-ws2:x+ws2+1]

ValueError: could not broadcast input array from shape (23,23,3) into shape (23,23)

In [0]:
image_data.reshape(SizeX, SizeY)

ValueError: cannot reshape array of size 989880 into shape (565,584)

# WORKING CODE for numpy array

In [0]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

img_data_array = np.arange(6500 * 4500, dtype=np.float32).reshape(6500, 4500)
img_strided = as_strided(image_data_array, shape=(6500 - 30, 4500 - 30, 31, 31),
                         strides=image_data.strides + image_data.strides, writeable=False)
# img_strided.shape == (6470, 4470, 31, 31)
for img_patch in img_strided.reshape(-1, 31, 31):
    print(img_patches.shape)
    # img_patch.shape == (patch_height, patch_width)

ValueError: mismatch in length of strides and shape

# WORKING CODE TO EXTRACT PATCHES


In [0]:
import os
import numpy as np
import scipy.ndimage as ndimage
import matplotlib
import matplotlib.pyplot as plt

def frac_eq_to(image, value=0):
    return (image == value).sum() / float(np.prod(image.shape))

def extract_patches(image, patchshape, overlap_allowed=1, cropvalue=None,
                    crop_fraction_allowed=0.1):
    """
    Given an image, extract patches of a given shape with a certain
    amount of allowed overlap between patches, using a heuristic to
    ensure maximum coverage.
    If cropvalue is specified, it is treated as a flag denoting a pixel
    that has been cropped. Patch will be rejected if it has more than
    crop_fraction_allowed * prod(patchshape) pixels equal to cropvalue.
    Likewise, patches will be rejected for having more overlap_allowed
    fraction of their pixels contained in a patch already selected.
    """
    jump_cols = int(patchshape[1] * overlap_allowed)
    jump_rows = int(patchshape[0] * overlap_allowed)
    
    # Restrict ourselves to the rectangle containing non-cropped pixels
    if cropvalue is not None:
        rows, cols = np.where(image != cropvalue)
        rows.sort(); cols.sort()
        active =  image[rows[0]:rows[-1], cols[0]:cols[-1]]
    else:
        active = image

    rowstart = 0; colstart = 0

    # Array tracking where we've already taken patches.
    covered = np.zeros(active.shape, dtype=bool)
    patches = []

    while rowstart < active.shape[0] - patchshape[0]:
        # Record whether or not e've found a patch in this row, 
        # so we know whether to skip ahead.
        got_a_patch_this_row = False
        colstart = 0
        while colstart < active.shape[1] - patchshape[1]:
            # Slice tuple indexing the region of our proposed patch
            region = (slice(rowstart, rowstart + patchshape[0]),
                      slice(colstart, colstart + patchshape[1]))
            
            # The actual pixels in that region.
            patch = active[region]

            # The current mask value for that region.
            cover_p = covered[region]
            if cropvalue is None or \
               frac_eq_to(patch, cropvalue) <= crop_fraction_allowed and \
               frac_eq_to(cover_p, True) <= overlap_allowed:
                # Accept the patch.
                patches.append(patch)
                
                # Mask the area.
                covered[region] = True
                
                # Jump ahead in the x direction.
                colstart += jump_cols
                got_a_patch_this_row = True
                #print "Got a patch at %d, %d" % (rowstart, colstart)
            else:
                # Otherwise, shift window across by one pixel.
                colstart += 1

        if got_a_patch_this_row:
            # Jump ahead in the y direction.
            rowstart += jump_rows
        else:
            # Otherwise, shift the window down by one pixel.
            rowstart += 1

    # Return a 3D array of the patches with the patch index as the first
    # dimension (so that patch pixels stay contiguous in memory, in a 
    # C-ordered array).
    return np.concatenate([pat[np.newaxis, ...] for pat in patches], axis=0)

def plot_patches(patches, fignum=None, low=0, high=0):
    """
    Given a stack of 2D patches indexed by the first dimension, plot the
    patches in subplots. 
    'low' and 'high' are optional arguments to control which patches
    actually get plotted. 'fignum' chooses the figure to plot in.
    """
    try:
        istate = plt.isinteractive()
        plt.ioff()
        if fignum is None:
            fig = plt.gcf()
        else:
            fig = plt.figure(fignum)
        if high == 0:
            high = len(patches)
        pmin, pmax = patches.min(), patches.max()
        dims = np.ceil(np.sqrt(high - low))
        for idx in range(high - low):
            spl = plt.subplot(dims, dims, idx + 1)
            ax = plt.axis('off')
            im = plt.imshow(patches[idx], cmap=matplotlib.cm.gray)
            cl = plt.clim(pmin, pmax)
        plt.show()
    finally:
        plt.interactive(istate)

def filter_patches(patches, min_mean=0.0, min_std=0.0):
    """
    Filter patches by some criterion on their mean and variance.
    
    Takes patches, a 3-dimensional stack of image patches (where
    the first dimension indexes the patch), and a minimum
    mean and standard deviation. Returns a stack of all the 
    patches that satisfy both of these criteria.
    """
    patchdim = np.prod(patches.shape[1:])
    patchvectors = patches.reshape(patches.shape[0], patchdim)
    means = patchvectors.mean(axis=1)
    stdevs = patchvectors.std(axis=1)
    indices = (means > min_mean) & (stdevs > min_std)
    return patches[indices]

def extract_patches_from_dir(directory, patchsize, 
                             smoothing=None, overlap_allowed=0.5,
                             cropvalue=None, crop_fraction_allowed=0.1,
                             min_mean=0, min_std=0):
    """
    Extract patches from an entire directory of images.
    
    If `smoothing` is not None, it is used as the standard deviation of a
    Gaussian filter applied to the image before extracting patches.
    
    `patchsize`, `overlap_allowed`, `cropvalue` and `crop_fraction_allowed`
    are passed along to `extract_patches()`. `min_mean` and `min_std` are
    passed along to `filter_patches()`.
    """
    output = {}
    for fname in os.listdir(directory):
        if fname[-4:] == '.png':
            outname = fname.replace('.','_').replace('-','_')
            assert outname not in output
            image = plt.imread(os.path.join(directory, fname))
            if smoothing is not None:
                image = ndimage.gaussian_filter(image, smoothing)
            # Extract patches from the image.
            output[outname] = extract_patches(image, patchsize,
                                              overlap_allowed,
                                              cropvalue, crop_fraction_allowed)

            # Filter the patches that don't meet our standards.
            output[outname] = filter_patches(output[outname], min_std=min_std,
                                             min_mean=min_mean)
    return output

In [0]:
pat = (23,23)
patches = extract_patches(image_data, pat, 0.1)

In [0]:
patches[0].shape

(23, 23, 3)

In [0]:
plot_patches(patches)

KeyboardInterrupt: 

In [0]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

img = np.arange(650 * 450, dtype=np.float32).reshape(650, 450)
img_strided = as_strided(img, shape=(650 - 30, 450 - 30, 31, 31),
                         strides=img.strides + img.strides, writeable=False)
# img_strided.shape == (6470, 4470, 31, 31)
for img_patch in img_strided.reshape(-1, 31, 31):
    print(img_patch)

By Sir

In [0]:
def create_patch(imgs,mod_image_2,mod_image_4,man,patch_size):
    ps=patch_size//2
    image_patches=[]
    for k in range(0,20):
        padded_imgs=np.pad(imgs[k],pad_width=ps,mode='edge')
        # padded_imgs_mod_2=np.pad(mod_image_2[k],pad_width=ps,mode='edge')
        # padded_imgs_mod_4=np.pad(mod_image_4[k],pad_width=ps,mode='edge')
        for i in range(0,imgs[k].shape[0]):
            for j in range(0,imgs[k].shape[1]):
                flatten_imgs=padded_imgs[i:i+patch_size,j:j+patch_size].ravel()
                # flatten_mod_2=padded_imgs_mod_2[i:i+patch_size,j:j+patch_size].ravel()
                # flatten_mod_4=padded_imgs_mod_4[i:i+patch_size,j:j+patch_size].ravel()
                imgs_mod_flatten=np.hstack((flatten_imgs))
                image_patches.append(imgs_mod_flatten)
    image_patches=np.vstack(image_patches)
    return image_patches

In [1]:
def imagetensor(imagedir):
    for i, im in tqdm(enumerate(os.listdir(imagedir))):
        image= Image.open(im)
        image= image.convert('HSV')
        if i == 0:
            images= np.expand_dims(np.array(image, dtype= float)/255, axis= 0)
        else:
            image= np.expand_dims(np.array(image, dtype= float)/255, axis= 0)
            images= np.append(images, image, axis= 0)
    return images

In [2]:
imagetensor('D:\google drive\Missions\mission Retina\navdeep sir datasets\DRIVE\training\images')

NameError: name 'tqdm' is not defined